# **Features engineering**

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [9]:
train = pd.read_csv(r'data/train_final.csv')

## 🚗 Creación de la variable `HasGarage`

Para reflejar explícitamente si una propiedad cuenta con garage o no, se creó la variable binaria `HasGarage`. Esta variable se construyó a partir de características físicas objetivas:

- `GarageArea` > 0
- `GarageCars` > 0

Si alguna de estas condiciones se cumple, `HasGarage = 1`, en caso contrario, `HasGarage = 0`.

Esta variable será útil para modelado, especialmente al combinar con transformaciones polinómicas o análisis de interacciones.

In [10]:
# Crear columna binaria HasGarage basada en si tiene espacio o autos asignados
train['HasGarage'] = (
    (train['GarageArea'] > 0) | 
    (train['GarageCars'] > 0)
).astype(int)


In [11]:
# Filtrar registros sin garage (HasGarage == 0)
sin_garage = train[train['HasGarage'] == 0]

# Verificar que GarageArea, GarageCars y GarageYrBlt sean cero
print("✅ Verificación de registros sin garage:")
print("GarageArea > 0:", (sin_garage['GarageArea'] > 0).sum())
print("GarageCars > 0:", (sin_garage['GarageCars'] > 0).sum())
print("GarageYrBlt > 0:", (sin_garage['GarageYrBlt'] > 0).sum())
print("Total de registros sin garage (HasGarage == 0):", sin_garage.shape[0])


✅ Verificación de registros sin garage:
GarageArea > 0: 0
GarageCars > 0: 0
GarageYrBlt > 0: 0
Total de registros sin garage (HasGarage == 0): 81


In [12]:
cor_gyb = train[train['HasGarage'] == 1][['GarageYrBlt', 'SalePrice']].corr().iloc[0, 1]
print(f"📌 Correlación filtrada GarageYrBlt vs SalePrice (solo con garage): {cor_gyb:.3f}")


📌 Correlación filtrada GarageYrBlt vs SalePrice (solo con garage): 0.548


## 📊 Selección de Variables Numéricas Fuertemente Correlacionadas con `SalePrice`

🎯 **Objetivo**: Seleccionar automáticamente las variables numéricas que presentan una alta correlación con `SalePrice`, para usarlas luego en una expansión polinómica con `PolynomialFeatures`.

📌 **Criterios de selección**:
- Variables **numéricas continuas** (float o int)
- Correlación absoluta con `SalePrice` **mayor a 0.4**
- Se excluyen columnas target, IDs y categóricas codificadas

In [13]:
# Asegurarse de tener solo variables numéricas
numeric_cols = train.select_dtypes(include=['int64', 'float64']).copy()

# Excluir la variable target y cualquier ID
excluded = ['SalePrice', 'Id']
numeric_cols = numeric_cols.drop(columns=[col for col in excluded if col in numeric_cols.columns])

# Calcular correlación con la variable target
correlations = numeric_cols.corrwith(train['SalePrice'])

# Filtrar por correlación fuerte
strong_features = correlations[correlations.abs() > 0.4].sort_values(ascending=False)

# Mostrar resultado
print("📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:")
print(strong_features)

# Guardar lista de features para usar en PolynomialFeatures
selected_features = strong_features.index.tolist()


📈 Variables numéricas con correlación fuerte (> 0.4) con SalePrice:
OverallQual          0.823505
GrLivArea            0.733793
GarageCars           0.685087
ExterQual            0.681659
GarageArea           0.668695
KitchenQual          0.668332
TotalBsmtSF          0.639330
BsmtQual             0.614882
GarageFinish         0.606831
1stFlrSF             0.606006
FullBath             0.598591
YearBuilt            0.588787
YearRemodAdd         0.568595
FireplaceQu          0.540808
Foundation_PConc     0.538138
TotRmsAbvGrd         0.536313
Fireplaces           0.484236
HeatingQC            0.477931
BsmtFinType1_GLQ     0.441554
MasVnrArea           0.429103
GarageType_Attchd    0.414782
dtype: float64


## 🔁 Expansión Polinómica de Features Seleccionadas

🎯 **Objetivo**: Generar nuevas variables que representen combinaciones cuadráticas entre las features más correlacionadas con `SalePrice`, usando `PolynomialFeatures`.

📌 **Características del proceso**:
- Se seleccionaron automáticamente variables numéricas con |correlación| > 0.4
- Se aplicó `PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)`
- Esto incluye:
  - Términos cuadráticos (e.g., `GrLivArea²`)
  - Interacciones cruzadas (e.g., `GrLivArea * GarageArea`)
- Se evita incluir un término constante (bias)

In [14]:
# Features seleccionadas
selected_features = strong_features.index.tolist()

# Agregar manualmente GarageYrBlt si no estaba incluida
if 'GarageYrBlt' not in selected_features:
    selected_features.append('GarageYrBlt')

# Subset del dataset con esas columnas
X_poly_base = train[selected_features]

# Crear objeto PolynomialFeatures (cuadrático)
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# Aplicar transformación
X_poly = poly.fit_transform(X_poly_base)

# Obtener nombres de las nuevas columnas
poly_feature_names = poly.get_feature_names_out(selected_features)

# Convertir a DataFrame para explorarlo
X_poly_df = pd.DataFrame(X_poly, columns=poly_feature_names)

# Mostrar forma
print(f"✅ Forma del dataset original: {X_poly_base.shape}")
print(f"📈 Forma del dataset expandido: {X_poly_df.shape}")


✅ Forma del dataset original: (1441, 22)
📈 Forma del dataset expandido: (1441, 275)


## 📐 Selección de Features Polinómicas con F-score

Se aplicó `PolynomialFeatures` (grado 2) a 22 variables altamente correlacionadas con `SalePrice`. Luego, se utilizó `SelectKBest` con `f_regression` para filtrar las variables polinómicas más explicativas.

- Total de variables generadas: 275  
- Umbral aplicado: **F_score > 1000**  
- Features seleccionadas: **115**

Esto permite conservar las combinaciones no lineales más informativas, reduciendo dimensionalidad y ruido en el modelado.


In [18]:
from sklearn.feature_selection import f_regression

# Obtener scores de F y p-values
F_scores, p_values = f_regression(X, y)

# Crear DataFrame para análisis
scores_df = pd.DataFrame({
    'Feature': X.columns,
    'F_score': F_scores,
    'p_value': p_values
})

# Filtrar por umbral (por ejemplo, F_score > 1000 o p < 0.05)
selected = scores_df[scores_df['F_score'] > 1000]  # ajustá el umbral a gusto

# Mostrar cuántas quedaron
print(f"Features seleccionadas: {selected.shape[0]}")

# Dataset final
X_filtered = X[selected['Feature'].values]


Features seleccionadas: 115


In [17]:
scores_df.sort_values(by="F_score").head(10)
scores_df["F_score"].describe()


count     275.000000
mean     1043.261630
std       688.390213
min       161.076961
25%       507.389334
50%       851.024202
75%      1342.491729
max      4000.682100
Name: F_score, dtype: float64

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Separar X e y
X = X_poly_df
y = train['SalePrice']

# Selección de features
selector = SelectKBest(score_func=f_regression, k=50)  # podés ajustar k
X_selected = selector.fit_transform(X, y)

# Obtener nombres de las columnas seleccionadas
selected_mask = selector.get_support()
selected_names = X.columns[selected_mask]

# Convertir a DataFrame
X_kbest_df = pd.DataFrame(X_selected, columns=selected_names)

# Confirmar forma
print(f"✅ Dataset final reducido con KBest: {X_kbest_df.shape}")

✅ Dataset final reducido con KBest: (1441, 50)


In [19]:
X_filtered.head()

,OverallQual,GrLivArea,GarageCars,ExterQual,GarageArea,KitchenQual,OverallQual^2,OverallQual GrLivArea,OverallQual GarageCars,OverallQual ExterQual,...,GarageFinish FullBath,GarageFinish TotRmsAbvGrd,GarageFinish HeatingQC,1stFlrSF FullBath,1stFlrSF YearBuilt,1stFlrSF YearRemodAdd,FullBath HeatingQC,FullBath GarageYrBlt,YearBuilt YearRemodAdd,TotRmsAbvGrd HeatingQC
0,7.0,7.444833,2.0,4.0,548.0,4.0,49.0,52.113833,14.0,28.0,...,4.0,16.0,10.0,13.506876,13527.136151,13527.136151,10.0,4006.0,4012009.0,40.0
1,6.0,7.141245,2.0,3.0,460.0,3.0,36.0,42.847471,12.0,18.0,...,4.0,12.0,10.0,14.282490,14111.100362,14111.100362,10.0,3952.0,3904576.0,30.0
2,7.0,7.488294,2.0,4.0,608.0,4.0,49.0,52.418055,14.0,28.0,...,4.0,12.0,10.0,13.650920,13657.745533,13664.570993,10.0,4002.0,4006002.0,30.0
3,7.0,7.448916,3.0,3.0,642.0,4.0,49.0,52.142413,21.0,21.0,...,1.0,7.0,4.0,6.869014,13154.162673,13531.958468,4.0,1998.0,3772550.0,28.0
4,8.0,7.695758,3.0,4.0,836.0,4.0,64.0,61.566064,24.0,32.0,...,4.0,18.0,10.0,14.088066,14088.065795,14088.065795,10.0,4000.0,4000000.0,45.0
